In [46]:
%pwd

'C:\\Code\\ditto_v2'

In [198]:
cd "C:\\Code\\ditto_v2"

C:\Code\ditto_v2


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [199]:
import geopandas as gpd
gdf_de = gpd.read_file('.\JupNB\DE_Data\VG250_GEM_WGS84.shp')

<>:2: SyntaxWarning: invalid escape sequence '\J'
<>:2: SyntaxWarning: invalid escape sequence '\J'
C:\Users\User\AppData\Local\Temp\ipykernel_16848\2888452935.py:2: SyntaxWarning: invalid escape sequence '\J'
  gdf_de = gpd.read_file('.\JupNB\DE_Data\VG250_GEM_WGS84.shp')


In [200]:
gdf_de.head()

,OBJID,BEGINN,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,...,NUTS,ARS_0,AGS_0,WSK,EWZ,KFL,DLM_ID,EPK,EPK_norm,geometry
0,DEBKGVG200000008,2022-12-20,6,4,1,010010000000,01001000,010010000000,Flensburg,Stadt,...,DEF01,010010000000,01001000,2008-01-01,92550,56,DEBKGDL20000E5MA,1652.678571,0.0,"POLYGON ((9.41266 54.82264, 9.41318 54.82124, ..."
1,DEBKGVG200000009,2022-12-20,6,4,1,010020000000,01002000,010020000000,Kiel,Stadt,...,DEF02,010020000000,01002000,2006-01-01,247717,118,DEBKGDL20000004J,2099.296610,0.0,"POLYGON ((10.16916 54.43138, 10.16957 54.43067..."
2,DEBKGVG20000000A,2022-12-20,6,4,1,010030000000,01003000,010030000000,Lübeck,Stadt,...,DEF03,010030000000,01003000,2006-02-01,218095,214,DEBKGDL20000DYMA,1019.135514,0.0,"POLYGON ((10.87684 53.98737, 10.87884 53.98595..."
3,DEBKGVG20000000B,2022-12-20,6,4,1,010040000000,01004000,010040000000,Neumünster,Stadt,...,DEF04,010040000000,01004000,1970-04-26,79502,71,DEBKGDL20000E4SA,1119.746479,0.0,"POLYGON ((9.99545 54.14972, 9.99713 54.14806, ..."
4,DEBKGVG20000000C,2021-11-17,6,4,1,010510011011,01051011,010510011011,Brunsbüttel,Stadt,...,DEF05,010510011011,01051011,2009-01-01,12603,65,DEBKGDL20000E2IK,193.892308,0.0,"POLYGON ((9.16439 53.94509, 9.16706 53.94302, ..."


In [201]:
from src.functionality_maps import Defs, paths
import pandas as pd

l_coverage = '20'
l_stores = ['sportscheck']
l_states = ['Sachsen']
radius_km = int(l_coverage)

print('Filtering stores...')
pdf_stores = pd.read_json(paths.stores, orient='records', lines=True)
pdf_stores = pdf_stores[pdf_stores['store'].isin(l_stores)]
print(f'  After filtering stores... pdf_stores:{pdf_stores.shape[0]}')
pdf_stores = pdf_stores[pdf_stores['GADM_1'].isin(l_states)]
print(f'  After filtering states... pdf_stores:{pdf_stores.shape[0]}\n')


print('Filtering geometry...')
l_bundeslaender = [Defs.dict_bundeslaender_id[x] for x in l_states]
print(f'  gdf_geom:{gdf_de.shape[0]}')
gdf_geom = gdf_de[gdf_de['SN_L'].isin(l_bundeslaender)]
print(f'  After filtering States, gdf_geom:{gdf_geom.shape[0]}')

print('Removing non-serializable columns...')
l_cols_non_serializable = ['WSK','BEGINN']
gdf_geom = gdf_geom.drop(l_cols_non_serializable,axis=1)

print('Computing areas...')
gdf_geom_6993 = gdf_geom.to_crs(epsg=6933)
gdf_geom_6993['KFL_GPD'] = round(gdf_geom_6993.geometry.area / 10**6, 2)
gdf_geom_4326 = gdf_geom.to_crs(epsg=4326)
gdf_geom_6993['lat'] = gdf_geom_4326.geometry.centroid.y
gdf_geom_6993['lon'] = gdf_geom_4326.geometry.centroid.x



Filtering stores...
  After filtering stores... pdf_stores:30
  After filtering states... pdf_stores:3

Filtering geometry...
  gdf_geom:10990
  After filtering States, gdf_geom:419
Removing non-serializable columns...
Computing areas...


C:\Users\User\AppData\Local\Temp\ipykernel_16848\4019681911.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_geom_6993['lat'] = gdf_geom_4326.geometry.centroid.y
C:\Users\User\AppData\Local\Temp\ipykernel_16848\4019681911.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_geom_6993['lon'] = gdf_geom_4326.geometry.centroid.x


In [202]:
gdf_circle.columns

Index(['geometry', 'KFL_GPD'], dtype='object')

In [203]:
l_center_points = list(zip(pdf_stores['lat'], pdf_stores['lon']))
len(l_center_points)

3

In [ ]:
import folium
from src.functionality_maps import f_maps
from shapely import geometry

l_center_points = list(zip(pdf_stores['lat'], pdf_stores['lon']))
l_fg = []
# l_fg_point, l_fg_circle, l_fg_merged = []
# l_fg_inside, l_fg_partly = []

fg_point = folium.FeatureGroup(name=f'Center points [{len(l_center_points)}]')
fg_circle = folium.FeatureGroup(name=f'Circles [r={radius_km} Km]')
fg_int = folium.FeatureGroup(name=f'Intersections')
fg_inside = folium.FeatureGroup(name=f'Markers - Inside towns')
fg_partly = folium.FeatureGroup(name=f'Markers - Partly towns')
fg_merged = folium.FeatureGroup(name=f'Circle - Summary inside')

for i,center_point in enumerate(l_center_points):
    lat = center_point[0]
    lon = center_point[1]

    center_point = geometry.Point(lon, lat)
    gdf_point = gpd.GeoDataFrame(geometry=[center_point], crs=f'EPSG:4326')
    fg_point = folium.FeatureGroup(name=f'Center point [lat={lat}, lon={lon}]')
    # fg_point = folium.FeatureGroup(name=f'Center point [lat={lat}, lon={lon}]')
    gjson_point = folium.Marker([center_point.y, center_point.x], 
                                icon=folium.Icon(color="red"),
                                tooltip=folium.features.GeoJsonTooltip(fields=['geometry'], values=['coordinates'])
                                )

    gjson_point.add_to(fg_point)
    # l_fg_point.append(fg_point)
    print(f'Processing [{i+1}/{len(l_center_points)}] {center_point}')
    ############# CIRCLE PROJECTION IN 4326 ####################
    # Convert the Shapely circle to a geoDataFrame
    poly_circle = f_maps.circle_around_lat_lon_point(lon=lon, lat=lat, radius=1000 * radius_km)
    gdf = gpd.GeoSeries([poly_circle])
    gdf_circle = gdf.to_frame(name='geometry').set_crs(epsg=4326)
    gdf_circle_6933 = gdf_circle.to_crs(epsg=6933)
    gdf_circle['KFL_GPD'] = round(gdf_circle_6933.geometry.area / 10**6, 2)

    # Generate the FeatureGroup and add it to the list
    # fg_circle = folium.FeatureGroup(name=f'Circle [lat={lat}, lon ={lon}, r={radius_km} Km]')
    gjson_circle = folium.GeoJson(gdf_circle, color='red')
    gjson_circle.add_to(fg_circle)
    # l_fg_circle.append(fg_circle)

    ############# CIRCLE PROJECTION IN 4326 ####################
    gdf_de_int = gpd.sjoin(gdf_geom_4326, gdf_circle, 
                        how="inner", predicate="intersects") #\
                            # .drop(l_cols_non_serializable, axis=1)
                            # 
    gjson = f_maps.get_folium_geojson(gdf_de_int, 
                                        fg_name=f'Intersection, [lat={lat}, lon={lon}, #={gdf_de_int.shape[0]}]',
                                        fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD', 'AGS',
                                                ],
                                        aliases = ['Name', 'Population', 'Area', 'Area_GPD','AGS',
                                                    ])  
    gjson.add_to(fg_int)
    # fgi = f_maps.get_folium_featuregroup(gdf_de_int, 
    #                                     fg_name=f'Intersection, [lat={lat}, lon={lon}, #={gdf_de_int.shape[0]}]',
    #                                     #  A={round(gdf_de_int.geometry.area.sum() / 10**6,2)} Km2, \                           
    #                                     #  A_data={gdf_de_int.KFL.sum()} Km2]',
    #                                     fields=['GEN', 'EWZ', 'KFL', 'KFL_GPD', 'AGS',
    #                                             #  'lat', 'lon',
    #                                             # 'distance_km'      
    #                                             ],
    #                                     aliases = ['Name', 'Population', 'Area', 'Area_GPD','AGS',
    #                                                         # 'lat', 'lon',                  
    #                                                         #    'Distance [Km]'          
    #                                                 ])  
    # l_fg.append(fgi)

    ############# UNION circle and gdf_geom ####################
    gdf_circle = gpd.overlay(gdf_geom_6993, gdf_circle_6933, how='intersection')
    poly_merged = gdf_circle.geometry.union_all()
    gdf = gpd.GeoSeries([poly_merged])
    gdf_merged = gdf.to_frame(name='geometry').set_crs(epsg=6933)



    ############# UNION circle and gdf_geom ####################
    gdf_circle['KFL_int'] = round(gdf_circle.geometry.area / 10**6, 2)
    gdf_circle['PERC_int'] = round(gdf_circle['KFL_int'] / gdf_circle['KFL_GPD'], 4)

    gdf_circle['EWZ_int'] = gdf_circle['EWZ'] * gdf_circle['PERC_int']
    gdf_circle['EWZ_int'] = gdf_circle['EWZ_int'].astype(int)

    gdf_circle['KFL_sum'] = gdf_circle['KFL_GPD']
    gdf_circle['KFL_per'] = round(gdf_circle['KFL_int'] / gdf_circle['KFL_sum'], 4)

    # ################### Distance to point of interest ###################
    gdf = pd.DataFrame(gdf_circle[['ARS','lat','lon','KFL_per']])
    gdf = gpd.GeoDataFrame(gdf, crs='epsg:4326', geometry=[geometry.Point(xy) for xy in zip(gdf['lon'], gdf['lat'])])
    print(f'lat={lat}, lon={lon}')
    gdf = f_maps.compute_dist_to_lat_lon(gdf, lon=center_point.y, lat=center_point.x, round_dec=2, units='km')

    #######################  Gemeinde Markers  ##########################
    gdf_inside = gdf[gdf['KFL_per'] == 1.0]
    gdf_partly = gdf[gdf['KFL_per'] != 1.0]
    # gdf_outside = compute_dist_to_lat_lon(gdf_outside,lat=center_point.y, lon=center_point.x, round_dec=2)
    
    
    
    # fg_inside = folium.FeatureGroup(name=f'Inside  [lat={lat}, lon ={lon}, {gdf_inside.shape[0]}]')
    for index, row in gdf_inside.iterrows():
        tooltip = folium.Tooltip(f"Distance[Km]: {row['dist']}")
        popup = folium.Popup(f"lat: {row['lat']}\nLon: {row['lon']}")
        folium.Marker(location=[row.lat, row.lon],
                      popup=popup,
                      icon=folium.Icon(color="darkgreen"),
                      tooltip=tooltip
                     ).add_to(fg_inside)
    # l_fg_inside.append(fg_inside)
    
    # fg_partly = folium.FeatureGroup(name=f'Partly  [lat={lat}, lon ={lon}, {gdf_partly.shape[0]}]')
    for index, row in gdf_partly.iterrows():
        tooltip = folium.Tooltip(f"Distance[Km]: {row['dist']}")
        popup = folium.Popup(f"lat: {row['lat']}\nLon: {row['lon']}")
        folium.Marker(location=[row.lat, row.lon],
                      popup=popup,
                      icon=folium.Icon(color="orange"),
                      tooltip=tooltip
                     ).add_to(fg_partly)
    # l_fg_partly.append(fg_partly)

    # fg_outside = folium.FeatureGroup(name=f'Outside  [lat={lat}, lon ={lon}, {gdf_outside.shape[0]}]')
    # for index, row in gdf_outside.iterrows():
    #     tooltip = folium.Tooltip(f"Distance[Km]: {row['dist']}")
    #     popup = folium.Popup(f"lat: {row['lat']}\nLon: {row['lon']}")
    #     folium.Marker(location=[row.lat, row.lon],
    #                   popup=popup,
    #                   icon=folium.Icon(color="red"),
    #                   tooltip=tooltip
    #                  ).add_to(fg_outside)
        
    gdf_merged['num_in'] = gdf_inside.shape[0]
    gdf_merged['num_part'] = gdf_partly.shape[0]
    gdf_merged['EWZ_int'] = gdf_circle['EWZ_int'].sum()
    gdf_merged['KFL_int'] = gdf_circle['KFL_int'].sum()

    gjson = f_maps.get_folium_geojson(
        gdf_merged, 
        fields=[
            'num_in', 'num_part', 
            'EWZ_int','KFL_int', 
            ],
        aliases = [
            '# Localities inside',
            '# Localities partially inside',
            'Population', 
            'Area [Km2]', 
            ])
    gjson.add_to(fg_merged)
    # fg_circle_merged = f_maps.get_folium_featuregroup(
    #     gdf_merged,
    #     fg_name = f'Merged Circle, [lat={lat}, lon ={lon}, r={radius_km} Km]',
    #     fields=[
    #         'num_in', 'num_part', 
    #         'EWZ_int','KFL_int', 
    #         ],
    #     aliases = [
    #         '# Localities inside',
    #         '# Localities partially inside',
    #         'Population', 
    #         'Area [Km2]', 
    #         ])
    # l_fg_merged.append(fg_circle_merged)


Processing [1/3] POINT (13.7354617 51.0484718)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return type(geom)(zip(*func(*zip(*geom.coords))))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\shapely\ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))


AttributeError: module 'src.functionality_maps.f_maps' has no attribute 'get_folium_geojson'

In [193]:
l_fg = [fg_point, fg_circle, fg_merged, fg_inside, fg_partly]

NameError: name 'fg_merged' is not defined

In [ ]:
import folium.map



m=folium.Map()
for fg in l_fg:
    fg.add_to(m)
folium.LayerControl(position='topright',collapsed=False).add_to(m)
# m.add_child(folium.LatLngPopup())

m.fit_bounds(m.get_bounds())
m

In [37]:
l_center_points

[(51.0484718, 13.7354617), (51.045427, 13.7359245), (51.3424406, 12.3732372)]